In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import numpy as np
import os
import pandas as pd
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI as OpenAI_llama
from llama_index.core.llms import ChatMessage
from pathlib import Path
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter, TokenTextSplitter
from llama_index.core.schema import TextNode
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core import PromptTemplate
from llama_index.core.schema import Document

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

client_llama = OpenAI_llama(model='gpt-4o-2024-05-13')

embed_model = OpenAIEmbedding(model="text-embedding-3-large", dimensions=128)
Settings.embed_model = embed_model

In [37]:
storage_context_N = StorageContext.from_defaults(persist_dir="./datamining_data/노직원전DB")
index_N = load_index_from_storage(storage_context_N)

storage_context_R = StorageContext.from_defaults(persist_dir="./datamining_data/롤스원전DB")
index_R = load_index_from_storage(storage_context_R)

storage_context_S = StorageContext.from_defaults(persist_dir="./datamining_data/싱어원전DB")
index_S = load_index_from_storage(storage_context_S)

storage_context_H = StorageContext.from_defaults(persist_dir="./datamining_data/홉스원전DB")
index_H = load_index_from_storage(storage_context_H)

storage_context_K = StorageContext.from_defaults(persist_dir="./datamining_data/칸트원전DB")
index_K = load_index_from_storage(storage_context_K)

retriever_R = VectorIndexRetriever(
    index=index_R,
    similarity_top_k=5
)

retriever_N = VectorIndexRetriever(
    index=index_N,
    similarity_top_k=5
)

retriever_K = VectorIndexRetriever(
    index=index_K,
    similarity_top_k=5
)

retriever_H = VectorIndexRetriever(
    index=index_H,
    similarity_top_k=5
)

retriever_S = VectorIndexRetriever(
    index=index_S,
    similarity_top_k=5
)

nodes_R = retriever_R.retrieve("시민 불복종의 주체는 체제의 합법성을 인정하는 시민인가?")

for node in nodes_R:
    print(node.text)    
    print("===\n\n")

우리는 정당화될 수 있는 시민 불복종은 일반적으로 정의감에 의해 상당한 정도로 규제되는 사회에서만 합당하고 효율적인 반대 형식이라는 사실을 인식해야 한다.
===


평등한 자유의 원칙에 대한 위반은 보다 적합한 시민 불복종의 대상이 된다. 이러한 원칙은 입법 체제에 있어서 평등한 시민권이 갖는 공통된 지위를 규정하며 정치 질서의 바탕에 깔려 있는 것이다. 그것이 충분히 지켜질 때 다른 부정의는 지속적이고 심각한 것일 수는 있어도 처리할수 없는 것은 아니라고 추정할 수 있다.
===


시민 불복종은 한편에서는 합법적인 테스트 케이스를 제기하는 것으로, 다른 한편에서는 양심적인 거부 및 여러 형식의 저항 간에 존재하는 것으로 정의되어 왔다. 이러한 가능성의 영역에 있어서 그것은 법에 대한 충실성의 경계에 있는 그러한 형태의 항의를 의미하는 것이다. 이렇게 이해하는 경우 시민 불복종은 전투적인 행위나 파괴로부터 분명히 구분되며 그것은 조직적인 강력한 저항과는 거리가 멀다.
===


내가[롤스가] 보기에는 시민 불복종의 문제는 어느 정도 정의로운 국가 내에서 그 체제의 합법성을 인정하고 받아들이는 시민들에 있어서만 생겨난다.
===


시민 불복종은 그것이 정치 권력을 쥐고 있는 다수자에게 제시된다는 의미에서뿐만 아니라 그것이 정치적 원칙, 즉 헌법과 사회 제도 일반을 규제하는 정의의 원칙들에 의해 지도되고 정당화되는 행위라는 의미에서 정치적 행위라는 점을 또한 주목해야 한다.
===




In [27]:
import pandas as pd

def find_id_by_text(file_path, search_text):
    df = pd.read_csv(file_path)
    
    row_index = df[df.apply(lambda row: row.astype(str).str.contains(search_text).any(), axis=1)].index
    
    if not row_index.empty:
        id_value = df.at[row_index[0], 'ID 소번호']
        return id_value
    else:
        return "Text not found in the document."

file_path_R = './datamining_data/롤스_원전.csv'
search_text = "우리는 정당화될 수 있는 시민 불복종은 일반적으로 정의감에 의해 상당한 정도로 규제되는 사회에서만 합당하고 효율적인 반대 형식이라는 사실을 인식해야 한다."
id_value = find_id_by_text(file_path_R, search_text)
id_value

'O-1-1-34-2'

In [28]:
for node in nodes_R:
    node_id = find_id_by_text(file_path_R, node.text)
    print(node_id)

O-1-1-34-2
O-1-1-23
O-1-1-18
Text not found in the document.
O-1-1-15-1


In [46]:
query_engine_R = index_R.as_query_engine()
response = query_engine_R.query('우리는 정당화될 수 있는 시민 불복종은 일반적으로 정의감에 의해 상당한 정도로 규제되는 사회에서만 합당하고 효율적인 반대 형식이라는 사실을 인식해야 한다.')

In [47]:
for node in response.source_nodes:
    print(f"Text: {node.text}, Similarity Score: {node.score}")

Text: 우리는 정당화될 수 있는 시민 불복종은 일반적으로 정의감에 의해 상당한 정도로 규제되는 사회에서만 합당하고 효율적인 반대 형식이라는 사실을 인식해야 한다., Similarity Score: 0.9996133857304029
Text: 법정은 항의자들의 행위가 갖는 시민 불복종성을 고려해야 하고, 그것이 헌법의 바탕을 이루는 정치적 원칙에 의해 정당화될 수 있다(혹은 됨직하다)는 사실을 참작해야 하며, 이러한 근거로 해서 법적인 제재를 줄여주거나 어떤 경우에는 보류해야 한다. 그러나 이와 같이 필요한 배경들을 결여할 경우에는 전혀 반대 현상이 일어날 수도 있다., Similarity Score: 0.7545767535461909


In [48]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine_R = RetrieverQueryEngine(retriever=retriever_R)

In [52]:
response_R = query_engine_R.query('시민 불복종의 주체는 체제의 합법성을 인정하는 시민인가?')

for node in response_R.source_nodes:
    print(f"Text: {node.text}, Similarity Score: {node.score}")

Text: 우리는 정당화될 수 있는 시민 불복종은 일반적으로 정의감에 의해 상당한 정도로 규제되는 사회에서만 합당하고 효율적인 반대 형식이라는 사실을 인식해야 한다., Similarity Score: 0.7917202493731996
Text: 평등한 자유의 원칙에 대한 위반은 보다 적합한 시민 불복종의 대상이 된다. 이러한 원칙은 입법 체제에 있어서 평등한 시민권이 갖는 공통된 지위를 규정하며 정치 질서의 바탕에 깔려 있는 것이다. 그것이 충분히 지켜질 때 다른 부정의는 지속적이고 심각한 것일 수는 있어도 처리할수 없는 것은 아니라고 추정할 수 있다., Similarity Score: 0.7864242405213867
Text: 시민 불복종은 한편에서는 합법적인 테스트 케이스를 제기하는 것으로, 다른 한편에서는 양심적인 거부 및 여러 형식의 저항 간에 존재하는 것으로 정의되어 왔다. 이러한 가능성의 영역에 있어서 그것은 법에 대한 충실성의 경계에 있는 그러한 형태의 항의를 의미하는 것이다. 이렇게 이해하는 경우 시민 불복종은 전투적인 행위나 파괴로부터 분명히 구분되며 그것은 조직적인 강력한 저항과는 거리가 멀다., Similarity Score: 0.7714097460361512
Text: 내가[롤스가] 보기에는 시민 불복종의 문제는 어느 정도 정의로운 국가 내에서 그 체제의 합법성을 인정하고 받아들이는 시민들에 있어서만 생겨난다., Similarity Score: 0.7328245991205599
Text: 시민 불복종은 그것이 정치 권력을 쥐고 있는 다수자에게 제시된다는 의미에서뿐만 아니라 그것이 정치적 원칙, 즉 헌법과 사회 제도 일반을 규제하는 정의의 원칙들에 의해 지도되고 정당화되는 행위라는 의미에서 정치적 행위라는 점을 또한 주목해야 한다., Similarity Score: 0.7215085818098889


In [62]:
nodes_R = retriever_R.retrieve("시민 불복종의 주체는 체제의 합법성을 인정하는 시민인가?")

for node in nodes_R:
    print(node.text, node.score)    
    print("===\n\n")

우리는 정당화될 수 있는 시민 불복종은 일반적으로 정의감에 의해 상당한 정도로 규제되는 사회에서만 합당하고 효율적인 반대 형식이라는 사실을 인식해야 한다. 0.7917017888880932
===


평등한 자유의 원칙에 대한 위반은 보다 적합한 시민 불복종의 대상이 된다. 이러한 원칙은 입법 체제에 있어서 평등한 시민권이 갖는 공통된 지위를 규정하며 정치 질서의 바탕에 깔려 있는 것이다. 그것이 충분히 지켜질 때 다른 부정의는 지속적이고 심각한 것일 수는 있어도 처리할수 없는 것은 아니라고 추정할 수 있다. 0.7864334913650654
===


시민 불복종은 한편에서는 합법적인 테스트 케이스를 제기하는 것으로, 다른 한편에서는 양심적인 거부 및 여러 형식의 저항 간에 존재하는 것으로 정의되어 왔다. 이러한 가능성의 영역에 있어서 그것은 법에 대한 충실성의 경계에 있는 그러한 형태의 항의를 의미하는 것이다. 이렇게 이해하는 경우 시민 불복종은 전투적인 행위나 파괴로부터 분명히 구분되며 그것은 조직적인 강력한 저항과는 거리가 멀다. 0.7714519128444942
===


내가[롤스가] 보기에는 시민 불복종의 문제는 어느 정도 정의로운 국가 내에서 그 체제의 합법성을 인정하고 받아들이는 시민들에 있어서만 생겨난다. 0.7327932020577351
===


시민 불복종은 그것이 정치 권력을 쥐고 있는 다수자에게 제시된다는 의미에서뿐만 아니라 그것이 정치적 원칙, 즉 헌법과 사회 제도 일반을 규제하는 정의의 원칙들에 의해 지도되고 정당화되는 행위라는 의미에서 정치적 행위라는 점을 또한 주목해야 한다. 0.7215474458837376
===




In [63]:
nodes_N = retriever_N.retrieve("시민 불복종의 주체는 체제의 합법성을 인정하는 시민인가?")

for node in nodes_N:
    print(node.text, node.score)    
    print("===\n\n")

우리의 삶보다 타인의 삶을 도덕적으로 보다 중요한 것으로 간주하여, 보다 큰 전반적인 사회적 선을 도모하려 할 수 없다. 우리 중 일부가 타인들을 위해 희생되는 것은 여하한 경우에도 정당화되지 않는다. 이 근원적인 생각, 즉 존재하는 것은 서로 독립적인 삶을 영위하는 서로 다른 개인들이며, 누구나 타인을 위해 희생되어서는 안 된다는 생각은 도덕적 측면 제약 사항들의 기초를 이루며, 이는 또한, 내 생각으로는, 타인에 대한 공격을 금지하는 자유주의적 측면 제약 사항에 귀결된다. 0.5705279819478456
===


개인들의 자연적 자산들이 도덕적 관점에서 볼 때 자의적이건 아니건 여하간에, 그 개인들은 그들에 대한 소유 권리를 지니며, 이로부터 유출되는 것에 대해서도 그러하다. 0.5177512655928013
===


그렇다면, 비슷하게, 왜 일부의 사람들이 전체적인 사회적 선을 위해서 비용을 부담하여 다른 사람들을 도와야 한다고 생각할 수 없는가? (...) 한 인격을 이런 식으로 이용하는 것은 그가 독립된 인격이라는 사실을 충분히 존중하지도, 고려에 넣지도 않는 것이다. 그는 자신의 희생에 대응하는 대가를 얻지 못하며, 누구도 그에게 희생을 강요할 권리가 없다. 0.5151218556099875
===


공리주의는 권리와 이의 비침해에 대해 응분의 고려를 하지 않고, 그 대신 부차적인 위치에 내버려 둔다는 것이다. (...) 일단의 군중이 어떤 범죄에 분노하여 범인을 찾느라 마을의 일부를 휩쓸며 살인하고 방화하면, 그곳에 사는 사람들의 권리가 침해받게 될 것이다. 해서 어떤 사람이 그 군중을 분노케 한 범죄의 혐의가 없다고 자신이 알고 있는 사람을 처벌하고, 이 처벌을 통해 타인에 의한 보다 큰 권리의 침해를 막을 수 있고, 해서 그 사회내에서의 권리 침해를 최소화할 수 있음을 들어 그 처벌을 정당화하려 할 수도 있다. 0.492660953413681
===


대부분의 사람들은 현재 시간 단면 원리들이 분배분에 관한 모든 것을 설명한다고 생각하

In [41]:
# from llama_index.core.postprocessor import SentenceTransformerRerank

# retriever_R_ = VectorIndexRetriever(index=index_R, similarity_top_k=10)
# reranker = SentenceTransformerRerank(model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=5)
# query_engine = RetrieverQueryEngine(retriever=retriever_R_, node_postprocessors=[reranker])
# response = query_engine.query("시민 불복종의 주체는 체제의 합법성을 인정하는 시민인가??")
# print(response)

# for response in response.source_nodes:
#     print(response.text, response.score)


/Users/gangjimin/Documents/GitHub/datamining/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/gangjimin/Documents/GitHub/datamining/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


시민 불복종의 주체는 체제의 합법성을 인정하는 시민이 아닐 수 있다.
시민 불복종에 대한 또 하나의 조건은 다음과 같은 것이다. 우리는 보통 정치적 다수자에게 정상적인 호소를 성실하게 해왔지만 그것이 성공적이지 않은 경우를 생각해 볼 수 있다. 합법적인 보상 수단은 아무런 소용도 없음이 판명된다. 그래서 예를 들어 현존하는· 정당이 소수자의 요구에 대해 그들 스스로 무관심을 나타내고 기꺼이 그 편의를 도모해주려는 의사를 보이지 않는 경우가 있다. 법을 폐기시키려는 시도는 무시되고 합법적인 항거와 시위가 성공을 거두지 못하게 된다. 시민 불복종은 최후의 대책이기 때문에 우리는 그것이 필요한 것임을 확신해야 한다. 11.704101
시민 불복종은 법에 분명히 반하는 것이긴 하지만 법에의 충실성과 민주 체제의 기본적인 정치적 원리들에 호소함을 나타내는 방식으로 합법적인 민주적 권위에 반대할 수 있는 근거를 정식화하려는 것이다. 그래서 우리는 그러한 반대의 지침이 되는 시민 불복종의 원리에 비추어 보아 입헌주의의 법적인 형태에 대해서 민주 헌법의 의도에 어긋나지 않는 어떤 불법적 항거에 가담할 수가 있다. 11.368902
시민 불복종은 한편에서는 합법적인 테스트 케이스를 제기하는 것으로, 다른 한편에서는 양심적인 거부 및 여러 형식의 저항 간에 존재하는 것으로 정의되어 왔다. 이러한 가능성의 영역에 있어서 그것은 법에 대한 충실성의 경계에 있는 그러한 형태의 항의를 의미하는 것이다. 이렇게 이해하는 경우 시민 불복종은 전투적인 행위나 파괴로부터 분명히 구분되며 그것은 조직적인 강력한 저항과는 거리가 멀다. 11.232321
시민 불복종을 정당화함에 있어서 우리는 어떤 개인적인 도덕 원칙이나 혹은 종교적 교설이 우리의 주장에 일치하고 이를 지지해준다고 해서 그것에 의거해서는 안 된다. 그리고 시민 불복종의 근거가 오직 개인이나 집단의 이익에만 기초할 수 없다는 것은 말할 필요도 없다. 그 대신 우리는 정치적인 질서의 바탕에 깔려 있는, 공유하고 있는 정의관에 의거하게

In [54]:
score_list = []
for response in response_R.source_nodes:
    score_list.append(response.score)
print(score_list)

[0.7917202493731996, 0.7864242405213867, 0.7714097460361512, 0.7328245991205599, 0.7215085818098889]


In [58]:
logits_R = np.array(score_list)

def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits))  # Subtract max for numerical stability
    return exp_logits / np.sum(exp_logits)

probabilities = softmax(logits_R)

print(probabilities)

[0.20620198 0.20511282 0.20205616 0.19440829 0.19222076]


In [59]:
from scipy.stats import entropy

def shannon_entropy(probabilities):
    return entropy(probabilities, base=2)

entropy_value = shannon_entropy(probabilities)
print(entropy_value)  # Output: Shannon entropy value


2.3213470965255913


In [61]:
score_R = 0
for element in score_list:
    score_R += element

print(score_R)

3.803887416861186
